<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Audio_Deepfake_Detection_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Audio Deepfake Detection, Fake Calls, Spoofing, Fraud Calls and Voice Cloning Analysis for Defensice Forensics**

In [ ]:
# ==============================================================================
# 0. Setup: Installations (Quiet)
# ==============================================================================
print("STEP 0: Installing required libraries...")
!pip install -q numpy librosa soundfile matplotlib IPython webrtcvad pydub noisereduce pyAudioAnalysis praat-parselmouth
!pip install -q speechbrain==0.5.14 torch==2.1.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118 # Pin versions for compatibility
!pip install -q moviepy Pillow transformers==4.39.3 datasets huggingface_hub # Pin transformers
!pip install -q openai langchain langchain_openai
!pip install -q groq langchain-groq
# Ensure vLLM version compatibility with Colab CUDA/PyTorch
# Check https://github.com/vllm-project/vllm/releases for recent versions suitable for CUDA 11.8/12.x
!pip install -q vllm==0.4.0.post1 # Adjust if necessary based on Colab environment
!pip install -q ipywidgets

print("Library installation complete.")

# ==============================================================================
# 1. Setup: Imports and Configuration
# ==============================================================================
print("\nSTEP 1: Importing libraries and configuring settings...")
import nest_asyncio
nest_asyncio.apply()

import os, json, asyncio, traceback, time, warnings, io
import numpy as np, librosa, librosa.display, soundfile as sf, matplotlib.pyplot as plt
import IPython.display as ipd
from IPython.display import display, clear_output, HTML
import webrtcvad
from pydub import AudioSegment
import noisereduce as nr
from pyAudioAnalysis import audioSegmentation as aS, ShortTermFeatures
import parselmouth
import torch
from concurrent.futures import ThreadPoolExecutor
from moviepy.editor import VideoFileClip
from PIL import Image
import ipywidgets as widgets
from google.colab import userdata, files
from datetime import datetime
import base64 # For embedding plots

# Filter common warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="Chunk.*not understood")
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU")
warnings.filterwarnings("ignore", message="Using a non-tuple sequence") # From HuggingFace/Datasets

# --- SpeechBrain ---
try:
    from speechbrain.inference.speaker import SpeakerRecognition
    from speechbrain.inference.classifiers import EncoderClassifier, LanguageIdentification
    from speechbrain.augment.time_domain import AddNoise
    SPEECHBRAIN_AVAILABLE = True
    print("SpeechBrain components imported.")
except ImportError as e:
    print(f"[Warning] Failed to import SpeechBrain: {e}. Related analyses will be skipped.")
    SPEECHBRAIN_AVAILABLE = False
    SpeakerRecognition = EncoderClassifier = LanguageIdentification = AddNoise = None

# --- LangChain & LLMs ---
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
# OpenAI (Optional Fallback)
OPENAI_ENABLED = False
try:
    from langchain_openai import OpenAI
    openai_api_key = userdata.get('OPENAI_API_KEY')
    if openai_api_key:
        os.environ['OPENAI_API_KEY'] = openai_api_key
        OPENAI_ENABLED = True
        print("OpenAI API Key loaded.")
    else:
        print("[Config] OpenAI API Key not found in Colab secrets.")
except ImportError: pass
except Exception as e: print(f"[Config Error] OpenAI: {e}")

# Groq (Primary for Reporting)
GROQ_ENABLED = False
try:
    from groq import Groq as GroqClient
    groq_api_key = userdata.get('GROQ_API_KEY')
    if groq_api_key:
        os.environ['GROQ_API_KEY'] = groq_api_key
        groq_client = GroqClient(api_key=groq_api_key)
        GROQ_ENABLED = True
        print("Groq API Key loaded and client initialized.")
    else:
        print("[Config] Groq API Key not found in Colab secrets. Reporting LLM disabled.")
except ImportError: print("[Config Error] groq library not installed.")
except Exception as e: print(f"[Config Error] Groq: {e}")

# --- vLLM ---
VLLM_AVAILABLE = False
try:
    if torch.cuda.is_available():
        from transformers import AutoTokenizer, AutoProcessor
        from vllm import LLM, EngineArgs, SamplingParams, RequestOutput, MultiModalData
        # Check CUDA capability for FP8 etc.
        capability = torch.cuda.get_device_capability()
        print(f"CUDA available. Device Capability: {capability}")
        if capability >= (8,0): print("   (Supports FP8 KV Cache)")
        VLLM_AVAILABLE = True
        print("vLLM components imported.")
    else:
        print("[Config Warning] CUDA not available. vLLM analysis will be skipped.")
except ImportError as e: print(f"[Config Error] Failed to import vLLM: {e}. vLLM analysis disabled.")
except Exception as e: print(f"[Config Error] vLLM Import: {e}. vLLM analysis disabled.")


# --- Global Settings ---
MAX_WORKERS = 4 # Keep low for Colab stability
executor = ThreadPoolExecutor(max_workers=MAX_WORKERS)
TARGET_SR = 16000 # Standard for speech models

print("Imports and configuration complete.")